In [ ]:
install.packages("tesseract")
install.packages("magick")
install.packages("stringr")
install.packages("pdftools")
library(magick)
library(tesseract)
library(stringr)
library(pdftools)

########################################################################################
#  Main driver - extracts data from a multi-page PDF into a dataframe
#  main function takes 3 arguments: path to PDF file, the page number of the first
#  page in the file, and the name of the county of the first page in the file
#  Note: on data loss or other error, program saves extracted data in a csv and
#        begins a new csv. Always need to check the end of prev. csv (where the error happened)
#        and beginning of new csv (where prev. state, county, and bank data may be incorrect)
#        there may also be data from prev. page on new csv!
#
#  IMPORTANT: ensure that working directory is Bank-data
#
#  TO RUN:
#  source("main.R")
#  main("test/1993B1_1.pdf", 1, "Fairfield")
#
########################################################################################

# format of final dataframe:  
# STATE | COUNTY | BANK | BRANCH | CITY | ZIP | IPC DEPOSITS | ALL OTHER DEPOSITS | TOTAL DEPOSITS

#source("ColumnCropping/cropColumns.R")
#source("dataProcessing.R")

# add a pageNum column to every table! for tracking purposes!

main <- function(pdfName, firstPage, firstCounty) {
    # read in image, set initial variables, create dataframe
    image <- image_read_pdf(pdfName, density = 600)
    pageNum <- firstPage
    state <- ""
    county <- firstCounty
    bank <- ""
    finalDF <- data.frame(matrix(ncol = 10, nrow = 0))
    colnames(finalDF) <- c("PageNum", "State", "County", "Bank", "Branch", "City", "ZIP", "IPC Deposits", "All Other Deposits", "Total Deposits")
    
    # create folder to store CSVs
    if (dir.exists("/home/idies/workspace/Storage/Casey/persistent/output")){
        unlink("/home/idies/workspace/Storage/Casey/persistent/output", recursive = TRUE)
    } 
    dir.create("/home/idies/workspace/Storage/Casey/persistent/output")
    
    # will iterate through each page, create a clean dataframe for it, and append it to final
    for (page in 1:length(image)) {
        print(paste0("Processing page ", pageNum, " ------------------------------------------"))
        # note that findCrop will always give you data that is a cont. of current county, or includes new county
        cropColumns <- cropColumns(image[page], pageNum)
        if (is.null(cropColumns[[1]]) & is.null(cropColumns[[2]])) { # indicates page cropping failed
            print(paste0("<<< failed to crop page ", pageNum, ", abandoning page and beginning new csv >>>"))
            write.csv(finalDF, paste0("/home/idies/workspace/Storage/Casey/persistent/output/pg", firstPage, "-", pageNum - 1, ".csv"), row.names = FALSE)
            firstPage <- pageNum + 1
            finalDF <- data.frame(matrix(ncol = 10, nrow = 0))
            colnames(finalDF) <- c("PageNum", "State", "County", "Bank", "Branch", "City", "ZIP", "IPC Deposits", "All Other Deposits", "Total Deposits")
            pageNum <- pageNum + 1
            next
        }
        state <- cropColumns[[1]]
        for (i in 2:length(cropColumns)) { # for each vector of extracted text (corresponding to a horizontal cropped section)
            columns <- cropColumns[[i]]
            splitColumns <- strsplit(columns, '\n')         # list of character vectors containing the entries of each column
            toAppend <- dataProcessing(splitColumns, county, state, bank)
            toAppend <- cbind(data.frame("PageNum" = rep(pageNum, nrow(toAppend))), toAppend)
            lastIndex <- nrow(toAppend)
            county <- toAppend$County[lastIndex] # reset the county and bank info for next img/page
            bank <- toAppend$Bank[lastIndex]
            finalDF <- rbind(finalDF, toAppend) # append new data to finalDF
            # check for data loss, stop if found
            for (col in 1:ncol(toAppend)) {
                lastVal <- toAppend[lastIndex, col]
                if (!is.na(lastVal) & lastVal == "#") {
                    write.csv(finalDF, paste0("/home/idies/workspace/Storage/Casey/persistent/output/pg", firstPage, "-", pageNum, ".csv"), row.names = FALSE)
                    #stop(paste0("probable data loss at page ", pageNum))
                    print(paste0("<<< probable data loss at page ", pageNum, ", beginning new csv >>>"))
                    firstPage <- pageNum
                    if (i == length(cropColumns)) {
                        firstPage <- pageNum + 1
                    }
                    finalDF <- data.frame(matrix(ncol = 10, nrow = 0))
                    colnames(finalDF) <- c("PageNum", "State", "County", "Bank", "Branch", "City", "ZIP", "IPC Deposits", "All Other Deposits", "Total Deposits")
                    break
                }
            }
        }

        pageNum <- pageNum + 1
    }
    
    #fileName <- tail(strsplit(pdfName, '/')[[1]], n = 1)
    #write.csv(finalDF, paste0(fileName, "data.csv"), row.names = FALSE)
    if (firstPage < pageNum) {
        write.csv(finalDF, paste0("/home/idies/workspace/Storage/Casey/persistent/output/pg", firstPage, "-", pageNum - 1, ".csv"), row.names = FALSE)
    }
}

# -------------------------------------------------------------------------------------------------------

NUM_COLUMNS <- 6
FIRST_NUMERIC_COL <- 4
LAST_NUMERIC_COL <- 6

dataProcessing <- function(splitColumns, origCounty, state, origBank) {
    
    # delete unneccessary whitespace data
    for (i in 1:NUM_COLUMNS) {
        col <- splitColumns[[i]]
        whiteSpaces <- col == ""
        splitColumns[[i]] <- splitColumns[[i]][!whiteSpaces]
    }
    
    # 2 cases
    #   1) data in splitColumns all belongs to origCounty
    #   2) data in splitColumns belongs to a new county that will be at the top of the data
    
    # check if this data belongs to origCounty or a new county!
    county <- origCounty
    columnOne <- splitColumns[[1]]
    indexParen1 <- grep("(", columnOne, fixed = TRUE)
    indexParen2 <- grep(")", columnOne, fixed = TRUE)
    indexOfCounty <- intersect(indexParen1, indexParen2)
    if (length(indexOfCounty) != 0) {
        indexOfCounty <- indexOfCounty[1]
        theoreticallyNewCountyLine <- strsplit(columnOne[indexOfCounty], " ")[[1]]
        indexOfParenString <- grep("(", theoreticallyNewCountyLine, fixed = TRUE)
        if (length(indexOfParenString) > 1) {
            theoreticallyNewCountyLine <- theoreticallyNewCountyLine[-indexOfParenString[1]]
            indexOfParenString <- indexOfParenString[-2]
        }
        parenString <- theoreticallyNewCountyLine[indexOfParenString] # edge case: tesseract reads County (003 )
        if (grep(")", theoreticallyNewCountyLine, fixed = TRUE) == indexOfParenString + 1){
            parenString <- paste0(parenString, theoreticallyNewCountyLine[indexOfParenString + 1])
        }
        if (str_length(parenString) == 5) {
            # print(paste0("length should be 5, deleting: ", parenString))
            tokens <- indexOfParenString:length(theoreticallyNewCountyLine)
            county <- paste(theoreticallyNewCountyLine[-tokens], collapse = " ") # possibly need to join for multi-word counties?
            deleteRows <- c(rep(TRUE, length(columnOne)))
            deleteRows[indexOfCounty] <- FALSE
            deleteRows[indexOfCounty + 1] <- FALSE # indexOfCounty can be a vector, so this deletes all!
            deleteRows[indexOfCounty - 1] <- FALSE
            splitColumns[[1]] <- columnOne[deleteRows]
        }
    }
    
    # all column data vectors need to have same length before putting in dataframe
    greatestLength <- 0
    for (i in 1:NUM_COLUMNS) {        # identify the column with the greatest length
        len <- length(splitColumns[[i]])
        if (len > greatestLength) {
            greatestLength <- len
        }
    }
    stringData <- data.frame(Temp = 1:greatestLength)  # dataframe shell
    for (i in 1:NUM_COLUMNS) {
        col = splitColumns[[i]]
        if (length(col) < greatestLength) {
            col <- c(col, rep('#', greatestLength - length(col))) # add '#' characters to fill in any short columns
        }
        stringData[paste0("Col", i)] <- col
    }
    stringData <- stringData[-1]

    
    # clean data - remove all periods/commas, replace all lone 'o' or 'O' with '0'
    stringData <- cleanData(stringData, FIRST_NUMERIC_COL, LAST_NUMERIC_COL)
    

    # separating bank and branch names into different columns - deletion part deactivated
    stringData <- separateBanksAndBranches(stringData, origBank)
    
    # add and fill county column
    stringData <- cbind(data.frame("County" = rep(county, nrow(stringData))), stringData)
    
    # add and fill state column
    stringData <- cbind(data.frame("State" = rep(state, nrow(stringData))), stringData)

    colnames(stringData) <- c("State", "County", "Bank", "Branch", "City", "ZIP", "IPC Deposits", "All Other Deposits", "Total Deposits")
    
    return(stringData)
}


# -------------------------------------------------------------------------------------------------------

# Given the first column as a character vector, searches for the start of a new county
# Returns a list containing:
#   vector of indexes of county change (or -1 if no new county)
#   vector of strings containing the name of the new counties
findCounty <- function(columnOne) {
    counties <- vector(mode = "character")
    index <- -1
    indexParen1 <- grep("(", columnOne, fixed = TRUE)
    indexParen2 <- grep(")", columnOne, fixed = TRUE)
    indexOfCounties <- intersect(indexParen1, indexParen2)
    if (length(indexOfCounties) != 0) {
        index <- indexOfCounties
        split <- strsplit(columnOne[indexOfCounties], " ")
        for (i in 1:length(indexOfCounties)) {
            counties <- c(counties, split[[i]][1])
        }
    }
    r <- list(index, counties)
    return(r)
}

# Given the first column as a character vector, searches for the totals table indicating the end of a county
# Returns index of first row in the table (or -1 if no county change)
findCountyTotals <- function(columnOne) {
    index <- -1
    indexOfTotals <- grep("COUNTY TOTALS", columnOne, fixed = TRUE)
    if (length(indexOfTotals) != 0) {
        index <- indexOfTotals
    }
    return(index)
}

# Cleans the numeric columns of dataframe of extracted text data and returns it
# Specifically, it deletes periods and commas and changes all lone 'o' and 'O's to '0's
cleanData <- function(dirtyDF, firstNumericCol, lastNumericCol) {
    for (i in 1:ncol(dirtyDF)) {
        dirtyDF[i] <- str_replace_all(dirtyDF[[i]], '[.]', ',')
        dirtyDF[i] <- str_replace_all(dirtyDF[[i]], '[,]', '')
        if (i >= firstNumericCol & i <= lastNumericCol) {
            dirtyDF[i] <- str_replace_all(dirtyDF[[i]], '[o]', '0')
            dirtyDF[i] <- str_replace_all(dirtyDF[[i]], '[O]', '0')
        }
    }
    return(dirtyDF)
}

# Given the dataframe of text data, separates banks and branch names into separate columns
# Bank name column appended to the beginning of the dataframe
separateBanksAndBranches <- function(stringData, origBank) {
    stringData <- cbind("Bank" = c(rep(NA, length(stringData$Col1))), stringData)
    curIndex <- 1
    for (i in stringData$Col1) {
        if (grepl("BANK", i) == TRUE | (grepl("MAIN OFFICE", i) == FALSE & grepl("BRANCH",i) == FALSE & grepl("FACILITY", i) == FALSE)) {
            stringData$Bank[curIndex] <- i    # copies bank name into new column
        }
        curIndex <- curIndex + 1
    }
    #assign the value of the bank name to each branch
#    curIndex <- 1
#    curBank <- origBank
#    for (i in stringData$Bank) {
#        if (is.na(i) == FALSE) {
#            curBank <- i
#        } else {
#            stringData$Bank[curIndex] <- curBank
#        }
#        curIndex <- curIndex + 1
#    }
    
    #remove bank names and only keep branch names in original first column
#    deleteRows <- c(rep(TRUE, length(stringData$Bank)))
#    count <- 1
#    for (i in 1:length(stringData$Bank)) {
#        if (grepl("MAIN OFFICE", stringData$Col1[i]) == FALSE & grepl("BRANCH", stringData$Col1[i]) == FALSE) {
#            deleteRows[i] <- FALSE
#        }
#    }
#    stringData <- stringData[deleteRows,]
    colnames(stringData)[2] <- "Branch"
    return(stringData)
}

# -------------------------------------------------------------------------------------------------------

cropColumns <- function(image, pageNum) {

    # stores percentage that each column should take up
    #col_pct <- c(0.36643, 0.15765, 0.09054, 0.12356, 0.09906, 0.04261)
    # original:
    #col_pct <- c(0.3863, 0.1608, 0.0906, 0.1202, 0.1127, 0.1158)
    # adjusted for SciServer
    col_pct <- c(0.418, 0.1608, 0.0906, 0.1202, 0.1127, 0.1158)
    
    # INITIAL BORDER CROP using cropPage function
    cropPageReturns <- cropPage(image, pageNum)
    if (is.null(cropPageReturns[[1]]) & is.null(cropPageReturns[[2]])) {
        return(cropPageReturns) # return null list to indicate that cropping failed
    }
    cropped <- cropPageReturns[[1]] # magick image of full cropped page
    returnList <- list(cropPageReturns[[2]]) # state for all data on this page
    # -------------------------------------------------------------------------
    # COMMENT 22-27, UNCOMMENT AND EDIT 31-32 FOR CROPPED SINGLE-PAGE READS
    # -----------------------------------------------------------------------
    # cropped <- image_read_pdf("1993B1_45.pdf", density = 600)     # put cropped filename here
    # returnList <- list("MASSACHUSETTS")                             # put state for that page here
    # CROP BY COUNTY SUMMARY TABLES 
    imgList <- findAndCrop(cropped, "COUNTY TOTALS")
    if (length(imgList) == 1 & is.null(imgList[[1]])) {
        print("no county summary tables on current page!")
        imgList <- list(cropped)
    } else {
        print(paste0("removed county summary tables, data from ", length(imgList), " counties remain"))
    }
    # CROP COLUMNS FOR EACH IMAGE
    for (i in 1:length(imgList)) {
        #image_write(imgList[[i]], path = paste0(pdfName, "/table", i, ".pdf"), format = "pdf")
        columns <- vector("character", 6) # will hold OCR-extracted text for each column
        width <- as.numeric(image_info(imgList[[i]])[2]) # set width and height for cropped page
        height <- as.numeric(image_info(imgList[[i]])[3])
        col_widths <- c(0, 0, 0, 0, 0, 0) # numeric vector to store pixel widths for each column
        for (j in 1:length(col_pct)) {
            col_widths[j] <- col_pct[j] * width
            # print(col_widths)
            if (j == length(col_pct)) {
                geo <- paste0(width - sum(col_widths[1:j-1]), 'x', height, '+', sum(col_widths[1:j-1]), '+', 0)
            } else {
                geo <- paste0(col_widths[j], 'x', height, '+', sum(col_widths[1:j-1]), '+', 0)
            }
            #geo <- paste0(col_widths[j], 'x', height, '+', sum(col_widths[1:j-1]), '+', 0)
            column <- image_crop(imgList[[i]], geo)
            #image_write(column, path = paste0(pageNum, "col", j, ".pdf"), format = "pdf")
            columns[j] <- ocr(column, engine = tesseract("eng"))
        }
        returnList <- c(returnList, list(columns))
    }
    return(returnList)
}

# -------------------------------------------------------------------------------------------------------

#   Function to crop the data table out of the given page
#   Parameters:
#       img - the magick image file (the return value of image_read_pdf)
#       page - the page number as a string
#   Returns
#       a list
#           1st element is the magick image file of the cropped page
#           2nd element is the state (as a string)
#
cropPage <- function(img, page) {
    width <- as.numeric(image_info(img)[2])
    height <- as.numeric(image_info(img)[3])                 
    crop_geo <- paste0(width*.94, 'x', height*0.93, '+', width*.06, '+', 0)
    croppedWatermark <- image_crop(img, crop_geo)
    startX <- 0
    while (startX <= image_info(croppedWatermark)[2]) {  # while startX less than width  
        strip_geo <- paste0(60, 'x', image_info(croppedWatermark)[3], '+', startX, '+', 0) # vertical strip of width 60 from startX
        strip <- image_crop(croppedWatermark, strip_geo)
        text <- ocr(strip, engine = tesseract("eng"))
        if (text != '') {
            break
        }
        startX <- startX + 30
    }
    startY <- 300
    while (startY <= image_info(croppedWatermark)[3]) {  # while startY less than height  
        strip_geo <- paste0(image_info(croppedWatermark)[2], 'x', 300, '+', 0, '+', startY - 300) # horizontal strip of height 300 from startY-300
        strip <- image_crop(croppedWatermark, strip_geo)
        text <- ocr(strip, engine = tesseract("eng"))
        if (grepl("DEPOSITS DEPOSITS", text, fixed = TRUE) == TRUE) {   # end of column headers, only look for 2 to minimize tesseract errors
            break
        }
        startY <- startY + 50
    }
    startY <- startY + 20
    cutFromRight <- 20
        while (cutFromRight <= image_info(croppedWatermark)[2] - startX) { # while cutFromRight less than new width 
        strip_geo <- paste0(160, 'x', image_info(croppedWatermark)[3], '+', image_info(croppedWatermark)[2] - cutFromRight - 160, '+', 0) # horizontal strip of height 100 from end-cutFromBottom
        strip <- image_crop(croppedWatermark, strip_geo)
        #image_write(strip, path = paste0("strip", cutFromRight, ".pdf"), format = "pdf")
        text <- ocr(strip, engine = tesseract("eng"))
        if (grepl("TED", text, fixed = TRUE) | grepl("ITS", text, fixed = TRUE) | grepl("SIT", text, fixed = TRUE)) {  
            break
        }
        cutFromRight <- cutFromRight + 60
    }
    if (cutFromRight - 60 > 0) {
        cutFromRight <- cutFromRight - 60
    }
    state <- "statePlaceholder"
    cutFromBottom <- 100
    while (cutFromBottom <= image_info(croppedWatermark)[3] - startY) {  # while cutFromBottom less than new height  
        strip_geo <- paste0(image_info(croppedWatermark)[2], 'x', 100, '+', 0, '+', image_info(croppedWatermark)[3] - cutFromBottom) # horizontal strip of height 100 from end-cutFromBottom
        strip <- image_crop(croppedWatermark, strip_geo)
        text <- ocr(strip, engine = tesseract("eng"))
        if (grepl(page, text, fixed = TRUE) == TRUE) { # found page number
            text <- strsplit(text, "\n")[[1]][1] 
            words <- strsplit(text, " ")[[1]]
            if (length(words) == 2) {
                state <- words[2]
            } else {
                state <- paste0(words[2], words[3])
            }
            # print(paste0("State: ", state))
            break
        }
        cutFromBottom <- cutFromBottom + 40
        if (cutFromBottom > 800) {
            print("WARNING: page number not detected, cropping failed")
            return(list(NULL, NULL)) # return null list to indicate that cropping failed
        }
    }
    crop_geo <- paste0(image_info(croppedWatermark)[2] - startX - cutFromRight, 'x', image_info(croppedWatermark)[3] - startY - cutFromBottom, '+', startX, '+', startY)
    finalCropped <- image_crop(croppedWatermark, crop_geo)
    print(paste0("cropped page dimensions: ", image_info(finalCropped)[2], "w x ", image_info(finalCropped)[3], "h"))
    if (image_info(finalCropped)[3] < 3000) {
        print("WARNING: cropped table unusually small, check for data loss")
    }
    returns <- list(finalCropped, state)
    return(returns)
}

# -------------------------------------------------------------------------------------------------------

SUMMARY_TABLE_HEIGHT <- 450 # height of county summary table, in pixels

findAndCrop <- function(img, phrase) {
    fullText <- ocr(img, engine = tesseract("eng"))
    fullText <- strsplit(fullText, "\n")[[1]]
    countyTables <- grep(phrase, fullText)
    numTables <- length(countyTables)
    if (numTables == 0) {
        return(list(NULL))
    }
    width <- as.numeric(image_info(img)[2])
    height <- as.numeric(image_info(img)[3])
    # print(paste0("width: ", width, "  height: ", height))
    # split whole image into top and bottom halves
    geo <- paste0(width, 'x', height/2, '+', 0, '+', 0)
    topHalf <- image_crop(img, geo)
    geo <- paste0(width, 'x', height/2, '+', 0, '+', height/2)
    botHalf <- image_crop(img, geo)
    topText <- ocr(topHalf, engine = tesseract("eng"))
    topText <- strsplit(topText, "\n")[[1]]
    botText <- ocr(img, engine = tesseract("eng"))
    botText <- strsplit(botText, "\n")[[1]]
    numTablesTop <- length(grep(phrase, topText))
    numTablesBot <- length(grep(phrase, botText))
    imgToSearch <- NULL
    botHalfIndicator <- FALSE
    if (numTablesTop + numTablesBot != numTables) { # maybe "COUNTY TOTALS" was on the halfway line, 
        #                                           search whole page? would be inefficient but a rare case
        imgToSearch <- img
    } else if (numTablesTop == 0) { # case that all tables are in bottom half
        imgToSearch <- botHalf
        height <- height/2
        botHalfIndicator <- TRUE
        print("searching bottom half for county summary table")
    } else if (numTablesBot == 0) { # case that all tables are in top half
        imgToSearch <- topHalf
        height <- height/2
        print("searching top half for county summary table")
    } else { # case that there are tables in both top and bottom halves, will search whole image
        imgToSearch <- img
    }
    yCursor <- 0 # will iterate down imgToSearch to find county tables
    tableLocations <- vector(mode = "numeric", length = numTables)
    for (i in 1:numTables) {
        # identify location of county table
        while (yCursor < height) {
            strip_geo <- paste0(width, 'x', 60, '+', 0, '+', yCursor) # crop geometry (type '?image_crop' for details)
            strip <- image_crop(imgToSearch, strip_geo)
            text <- ocr(strip, engine = tesseract("eng"))
            if (grepl(phrase, text, fixed = TRUE) == TRUE) {
                break
            }
            yCursor <- yCursor + 20
        } # now assuming yCursor points to the top of "COUNTY TOTALS"
        tableLocations[i] <- yCursor
        if (botHalfIndicator == TRUE) {
            tableLocations[i] <- tableLocations[i] + height
        }
        yCursor <- yCursor + SUMMARY_TABLE_HEIGHT 
    }
    # print("table locations: ")
    # print(tableLocations)
    imgList <- vector(mode = "list", length = numTables) # return list of images, will append last image if not blank
    top <- 0  # will track y-coord where current image should start
    for (i in 1:numTables) {
        yDim <- tableLocations[[i]]
        geo <- paste0(width, 'x', yDim - top, '+', 0, '+', top) # crop geometry (type '?image_crop' for details)
        cropped <- image_crop(img, geo)
        if (!(i == 1 & ocr(cropped, engine = tesseract("eng")) == "")) { # check that section above first county table isn't blank
            imgList[[i]] <- cropped 
        }
        top <- yDim + SUMMARY_TABLE_HEIGHT
    } # check that section after final county table isn't blank
    geo <- paste0(width, 'x', as.numeric(image_info(img)[3]) - top, '+', 0, '+', top) # crop geometry (type '?image_crop' for details)
    cropped <- image_crop(img, geo)    
    text <- ocr(cropped, engine = tesseract("eng"))
    if (text != "") {
        imgList <- c(imgList, cropped)
    }
    if (is.null(imgList[[1]])) {
        return(imgList[-1]) # case that county table was at top of page, first image was blank
    }
    #image_write(imgList[[i]], path ="findAndCrop/test.pdf", format = "pdf")
    return(imgList)
}

# -------------------------------------------------------------------------------------------------------

# main("pathToPDF", firstPageNum, "firstCountyName")
main("/home/idies/workspace/Storage/Casey/persistent/1993B5_201-end.pdf", 201, "McDonald")